<h2>Import dependencies</h2>

In [225]:
import pandas as pd
import numpy as np
import datetime # to handle date/time attributes
from os import listdir # os is a module for interacting with the OS
from os.path import isfile, join # to verify file object, and concatenate paths
import glob # to find pathnames matching a specific pattern
import re # regular expressions :)

# set seed for reproducibility
np.random.seed(0)

<h2>Import data</h2>

In [226]:
apps_df = pd.read_csv("./data/googleplaystore.csv")
apps_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,7-Jan-18,1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,15-Jan-18,2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,1-Aug-18,1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,8-Jun-18,Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,20-Jun-18,1.1,4.4 and up


<h2>Explore Data Types</h2>

In [227]:
print("Shape of data (rows,columns): ",apps_df.shape)
print(apps_df.dtypes)

Shape of data (rows,columns):  (10841, 13)
App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object


All of the columns are objects except for the rating column which is float. We will try to change this by changing some of the columns types to numeric ones, while others to strings.

<h3>Reviews</h3>

We check first if all of the values are actually numeric.

In [228]:
apps_df.Reviews.str.isnumeric().sum()

10840

There seems to be one value out of the 10841 values that is non-numeric.

In [229]:
apps_df[~apps_df.Reviews.str.isnumeric()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,11-Feb-18,1.0.19,4.0 and up,NaN


This app has a rating of 19.0, which doesn't make any sense. Also, its size, price and category attributes have weird values, therefore we should simply remove this entire row .

In [230]:
apps_df = apps_df[apps_df.Reviews.str.isnumeric()]

Finally, we convert the Reviews column's type to numeric

In [231]:
apps_df.Reviews=pd.to_numeric(apps_df.Reviews)
print(apps_df.dtypes)

App                object
Category           object
Rating            float64
Reviews             int64
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object


<h3>Size</h3>

The size attribute should also be numeric. 

In [232]:
apps_df.Size.value_counts()

Varies with device    1695
11M                    198
12M                    196
14M                    194
13M                    191
15M                    184
17M                    160
19M                    154
26M                    149
16M                    149
25M                    143
20M                    139
21M                    138
24M                    136
10M                    136
18M                    133
23M                    117
22M                    114
29M                    103
27M                     97
28M                     95
30M                     84
33M                     79
3.3M                    77
37M                     76
35M                     72
31M                     70
2.9M                    69
2.5M                    68
2.3M                    68
                      ... 
121k                     1
511k                     1
598k                     1
251k                     1
351k                     1
597k                     1
4

It seems that most of the values have either the suffix M or the suffix k. We could replace them by 10^6 and 10^3 respectively to make them numeric

In [233]:
apps_df.Size=apps_df.Size.str.replace('k','e+3')
apps_df.Size=apps_df.Size.str.replace('M','e+6')
apps_df.Size.head()

0     19e+6
1     14e+6
2    8.7e+6
3     25e+6
4    2.8e+6
Name: Size, dtype: object

We now make sure that all of the values are numeric before converting the entire column

In [234]:
def is_numeric(v):
    try:
        float(v)
        return True
    except ValueError:
        return False
    
temp=apps_df.Size.apply(lambda x: is_numeric(x))
apps_df.Size[~temp].value_counts()

Varies with device    1695
Name: Size, dtype: int64

There seems to be many rows that have size attributes with the values "Varies with device". We will simply change all of them to NaN

In [235]:
apps_df.Size=apps_df.Size.replace('Varies with device',np.nan)

Finally, we convert the column's type to numeric

In [236]:
apps_df.Size=pd.to_numeric(apps_df.Size)
print(apps_df.dtypes)

App                object
Category           object
Rating            float64
Reviews             int64
Size              float64
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object


<h3>Installs</h3>

We first check the unique values of this column.

In [237]:
apps_df.Installs.value_counts()

1,000,000+        1579
10,000,000+       1252
100,000+          1169
10,000+           1054
1,000+             907
5,000,000+         752
100+               719
500,000+           539
50,000+            479
5,000+             477
100,000,000+       409
10+                386
500+               330
50,000,000+        289
50+                205
5+                  82
500,000,000+        72
1+                  67
1,000,000,000+      58
0+                  14
0                    1
Name: Installs, dtype: int64

All of the values are either pure numbers or numbers prefixed with the sign '+'. We can convert the latter by simply removing the '+' sign.

In [238]:
apps_df.Installs=apps_df.Installs.apply(lambda x: x.strip('+'))
apps_df.Installs=apps_df.Installs.apply(lambda x: x.replace(',',''))
apps_df.Installs.value_counts()

1000000       1579
10000000      1252
100000        1169
10000         1054
1000           907
5000000        752
100            719
500000         539
50000          479
5000           477
100000000      409
10             386
500            330
50000000       289
50             205
5               82
500000000       72
1               67
1000000000      58
0               15
Name: Installs, dtype: int64

Now that everything seems to be in order, we can safetly convert this column's type to numeric.

In [239]:
apps_df.Installs=pd.to_numeric(apps_df.Installs)
print(apps_df.dtypes)

App                object
Category           object
Rating            float64
Reviews             int64
Size              float64
Installs            int64
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object


<h3>Price</h3>

We first get a feeling of the format of this column

In [240]:
apps_df.Price.value_counts()

0           10040
$0.99         148
$2.99         129
$1.99          73
$4.99          72
$3.99          63
$1.49          46
$5.99          30
$2.49          26
$9.99          21
$6.99          13
$399.99        12
$14.99         11
$4.49           9
$24.99          7
$29.99          7
$3.49           7
$7.99           7
$19.99          6
$5.49           6
$11.99          5
$8.99           5
$12.99          5
$6.49           5
$16.99          3
$10.00          3
$2.00           3
$1.00           3
$9.00           2
$8.49           2
            ...  
$3.08           1
$1.04           1
$1.26           1
$37.99          1
$299.99         1
$1.20           1
$25.99          1
$4.84           1
$1.59           1
$15.46          1
$74.99          1
$15.99          1
$109.99         1
$3.02           1
$3.90           1
$1.75           1
$18.99          1
$4.80           1
$3.28           1
$154.99         1
$3.61           1
$4.59           1
$1.29           1
$400.00         1
$394.99   

All of the values are either 0 or are numbers prefixed with the dollar sign. We will begin by removing the dollar signs.

In [241]:
apps_df.Price=apps_df.Price.apply(lambda x: x.strip('$'))
apps_df.Price.value_counts()

0          10040
0.99         148
2.99         129
1.99          73
4.99          72
3.99          63
1.49          46
5.99          30
2.49          26
9.99          21
6.99          13
399.99        12
14.99         11
4.49           9
3.49           7
7.99           7
29.99          7
24.99          7
5.49           6
19.99          6
6.49           5
12.99          5
8.99           5
11.99          5
10.00          3
16.99          3
2.00           3
1.00           3
10.99          2
3.95           2
           ...  
154.99         1
25.99          1
389.99         1
1.26           1
3.08           1
15.99          1
5.00           1
2.56           1
4.59           1
3.90           1
74.99          1
15.46          1
14.00          1
3.02           1
1.75           1
28.99          1
1.04           1
4.77           1
30.99          1
1.76           1
19.40          1
1.61           1
1.97           1
1.20           1
3.04           1
3.61           1
3.28           1
2.90          

Finally, we make sure that all of the values are numeric then convert the column's dtype

In [242]:
temp=apps_df.Price.apply(lambda x: is_numeric(x))
apps_df.Price[~temp].value_counts()

Series([], Name: Price, dtype: int64)

In [243]:
apps_df.Price=pd.to_numeric(apps_df.Price)
print(apps_df.dtypes)

App                object
Category           object
Rating            float64
Reviews             int64
Size              float64
Installs            int64
Type               object
Price             float64
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object


<h2>Duplicates</h2>

Check if all app names are unique

In [244]:
apps_df["App"].is_unique

False

In [245]:
duplicates = apps_df[apps_df["App"].isin(apps_df[apps_df.duplicated()]["App"])]
duplicates.sort_values("App", inplace = False) 

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
1393,10 Best Foods for You,HEALTH_AND_FITNESS,4.0,2490,3800000.0,500000,Free,0.00,Everyone 10+,Health & Fitness,17-Feb-17,1.9,2.3.3 and up
1407,10 Best Foods for You,HEALTH_AND_FITNESS,4.0,2490,3800000.0,500000,Free,0.00,Everyone 10+,Health & Fitness,17-Feb-17,1.9,2.3.3 and up
2543,1800 Contacts - Lens Store,MEDICAL,4.7,23160,26000000.0,1000000,Free,0.00,Everyone,Medical,27-Jul-18,7.4.1,5.0 and up
2322,1800 Contacts - Lens Store,MEDICAL,4.7,23160,26000000.0,1000000,Free,0.00,Everyone,Medical,27-Jul-18,7.4.1,5.0 and up
2256,2017 EMRA Antibiotic Guide,MEDICAL,4.4,12,3800000.0,1000,Paid,16.99,Everyone,Medical,27-Jan-17,1.0.5,4.0.3 and up
2385,2017 EMRA Antibiotic Guide,MEDICAL,4.4,12,3800000.0,1000,Paid,16.99,Everyone,Medical,27-Jan-17,1.0.5,4.0.3 and up
1337,21-Day Meditation Experience,HEALTH_AND_FITNESS,4.4,11506,15000000.0,100000,Free,0.00,Everyone,Health & Fitness,2-Aug-18,3.0.0,4.1 and up
1434,21-Day Meditation Experience,HEALTH_AND_FITNESS,4.4,11506,15000000.0,100000,Free,0.00,Everyone,Health & Fitness,2-Aug-18,3.0.0,4.1 and up
2522,420 BZ Budeze Delivery,MEDICAL,5.0,2,11000000.0,100,Free,0.00,Mature 17+,Medical,6-Jun-18,1.0.1,4.1 and up
7035,420 BZ Budeze Delivery,MEDICAL,5.0,2,11000000.0,100,Free,0.00,Mature 17+,Medical,6-Jun-18,1.0.1,4.1 and up


Since there are many apps that are repeated, we could keep the first of each duplicate app and remove the rest

In [246]:
apps_df.drop_duplicates(subset ="App",keep = "first", inplace = True) 
apps_df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000.0,10000,Free,0.0,Everyone,Art & Design,7-Jan-18,1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,15-Jan-18,2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000.0,5000000,Free,0.0,Everyone,Art & Design,1-Aug-18,1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000.0,50000000,Free,0.0,Teen,Art & Design,8-Jun-18,Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000.0,100000,Free,0.0,Everyone,Art & Design;Creativity,20-Jun-18,1.1,4.4 and up
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5600000.0,50000,Free,0.0,Everyone,Art & Design,26-Mar-17,1,2.3 and up
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19000000.0,50000,Free,0.0,Everyone,Art & Design,26-Apr-18,1.1,4.0.3 and up
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29000000.0,1000000,Free,0.0,Everyone,Art & Design,14-Jun-18,6.1.61.1,4.2 and up
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33000000.0,1000000,Free,0.0,Everyone,Art & Design,20-Sep-17,2.9.2,3.0 and up
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3100000.0,10000,Free,0.0,Everyone,Art & Design;Creativity,3-Jul-18,2.8,4.0.3 and up


<h2>Cleaning Data</h2>

We will now try to look for any corrupted/abnormal data and deal with it.

<h3>Category</h3>

We list all of the unique attributes and see if something doesn't fit

In [247]:
apps_df.Category.unique()

array(['ART_AND_DESIGN', 'AUTO_AND_VEHICLES', 'BEAUTY',
       'BOOKS_AND_REFERENCE', 'BUSINESS', 'COMICS', 'COMMUNICATION',
       'DATING', 'EDUCATION', 'ENTERTAINMENT', 'EVENTS', 'FINANCE',
       'FOOD_AND_DRINK', 'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME',
       'LIBRARIES_AND_DEMO', 'LIFESTYLE', 'GAME', 'FAMILY', 'MEDICAL',
       'SOCIAL', 'SHOPPING', 'PHOTOGRAPHY', 'SPORTS', 'TRAVEL_AND_LOCAL',
       'TOOLS', 'PERSONALIZATION', 'PRODUCTIVITY', 'PARENTING', 'WEATHER',
       'VIDEO_PLAYERS', 'NEWS_AND_MAGAZINES', 'MAPS_AND_NAVIGATION'],
      dtype=object)

Everything seems to be in order

<h3> Size </h3>

In [248]:
# Find the number of nan values
print("Nan Values = ", apps_df["Size"].isnull().sum())

Nan Values =  1227


Since we would need the size values for each application to formulate our questions, we would omit the nan values.

In [249]:
# assign the data frame with the values that does not contain null in the coulmn 'Size'
apps_df = apps_df[pd.notnull(apps_df['Size'])]

# Checking the result
print("Nan Values = ", apps_df["Size"].isnull().sum())

Nan Values =  0


<h3>Rating</h3>

In [250]:
print("Min rating = ",apps_df["Rating"].min())
print("Max rating = ",apps_df["Rating"].max())
print("Nan Values = ", apps_df["Rating"].isnull().sum())

Min rating =  1.0
Max rating =  5.0
Nan Values =  1405


All ratings seem to be within range, however some ratings has a nan value, which could mean that they are not rated yet. That we can consider them as 0 rating.

In [251]:
# making a copy of the data frame to avoid the SettingWithCopyWarning warning
apps_df = apps_df.copy()

# replacing the 'Rating' coulmn with the one with zeros instead of Nan
apps_df['Rating'] = apps_df['Rating'].fillna(0)

# Checking the result
print("Nan Values = ", apps_df["Rating"].isnull().sum())

Nan Values =  0


<h3>Type</h3>

In [252]:
apps_df.Type.value_counts()

Free    7747
Paid     685
Name: Type, dtype: int64

Nothing wrong here!

<h2>Content Rating</h2>

In [253]:
apps_df["Content Rating"].unique()

array(['Everyone', 'Teen', 'Everyone 10+', 'Mature 17+',
       'Adults only 18+', 'Unrated'], dtype=object)

We would represent every rating category by a number representation according to the following table :-

| Rating Category | # |
|-----------------|---|
| Everyone        | 1 |
| Teen            | 2 |
| Everyone 10+    | 3 |
| Mature 17+      | 4 |
| Adults only 18+ | 5 |
| Unrated         | 0 |

In [254]:
# We replace by creating a dictionary with the replacment as key/value pair, and pass it to the replace method.

replacing_Dict = { 'Everyone': 1, 'Teen': 2, 'Everyone 10+': 3, 'Mature 17+': 4, 'Adults only 18+': 5, 'Unrated': 0 }
apps_df["Content Rating"]=apps_df["Content Rating"].replace(to_replace=replacing_Dict)

<h2>Genres</h2>

In [255]:
apps_df.Genres.unique()

array(['Art & Design', 'Art & Design;Pretend Play',
       'Art & Design;Creativity', 'Art & Design;Action & Adventure',
       'Auto & Vehicles', 'Beauty', 'Books & Reference', 'Business',
       'Comics', 'Comics;Creativity', 'Communication', 'Dating',
       'Education', 'Education;Creativity', 'Education;Education',
       'Education;Action & Adventure', 'Education;Pretend Play',
       'Education;Brain Games', 'Entertainment',
       'Entertainment;Brain Games', 'Entertainment;Music & Video',
       'Events', 'Finance', 'Food & Drink', 'Health & Fitness',
       'House & Home', 'Libraries & Demo', 'Lifestyle',
       'Lifestyle;Pretend Play', 'Adventure;Action & Adventure', 'Arcade',
       'Casual', 'Card', 'Casual;Pretend Play', 'Strategy', 'Action',
       'Puzzle', 'Sports', 'Word', 'Racing', 'Casual;Creativity',
       'Simulation', 'Adventure', 'Board', 'Trivia', 'Role Playing',
       'Simulation;Education', 'Action;Action & Adventure',
       'Casual;Brain Games', 'Simulat

Some of the values here seem to be composed of multiple labels, which are separated by a semicolon (ex: 'Sports;Action & Adventure'). We can change divide this column into 2 columns: 'Primary_Genres' & 'Secondary_Genres'

In [256]:
# pd.concat([apps_df, apps_df["Genres"].str.split(';', expand=True)], axis=1)
split = apps_df["Genres"].str.split(';', expand=True)
# split = split.rename(columns={0:"Primary Genres",1:"Secondary Genres"})
split

,0,1
0,Art & Design,None
1,Art & Design,Pretend Play
2,Art & Design,None
3,Art & Design,None
4,Art & Design,Creativity
5,Art & Design,None
6,Art & Design,None
7,Art & Design,None
8,Art & Design,None
9,Art & Design,Creativity


In [257]:
apps_df = pd.concat([apps_df,split],axis=1)
apps_df = apps_df.rename(columns={0:"Primary Genres",1:"Secondary Genres"})
apps_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Primary Genres,Secondary Genres
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000.0,10000,Free,0.0,1,Art & Design,7-Jan-18,1.0.0,4.0.3 and up,Art & Design,None
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000.0,500000,Free,0.0,1,Art & Design;Pretend Play,15-Jan-18,2.0.0,4.0.3 and up,Art & Design,Pretend Play
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000.0,5000000,Free,0.0,1,Art & Design,1-Aug-18,1.2.4,4.0.3 and up,Art & Design,None
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000.0,50000000,Free,0.0,2,Art & Design,8-Jun-18,Varies with device,4.2 and up,Art & Design,None
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000.0,100000,Free,0.0,1,Art & Design;Creativity,20-Jun-18,1.1,4.4 and up,Art & Design,Creativity


<h3>Last Updated</h3>

In [258]:
apps_df["Last Updated"].unique()

array(['7-Jan-18', '15-Jan-18', '1-Aug-18', ..., '20-Jan-14', '16-Feb-14',
       '23-Mar-14'], dtype=object)

Nothing seems wrong here!

<h3>Current Ver</h3>

In [259]:
apps_df["Current Ver"].unique()

array(['1.0.0', '2.0.0', '1.2.4', ..., '1.0.612928', '0.3.4', '2.0.148.0'],
      dtype=object)

Aside from the fact that some values have the vale 'Varies with device', nothing seems to be wrong.

<h3>Android Ver</h3>

In [260]:
apps_df["Android Ver"].unique()

array(['4.0.3 and up', '4.2 and up', '4.4 and up', '2.3 and up',
       '3.0 and up', '4.1 and up', '4.0 and up', '2.2 and up',
       '5.0 and up', '6.0 and up', '1.6 and up', '2.1 and up',
       '5.1 and up', '1.5 and up', '7.0 and up', '4.3 and up',
       '4.0.3 - 7.1.1', '2.0 and up', '2.3.3 and up',
       'Varies with device', '3.2 and up', '4.4W and up', '7.1 and up',
       '7.0 - 7.1.1', '8.0 and up', '5.0 - 8.0', '3.1 and up',
       '2.0.1 and up', '4.1 - 7.1.1', nan, '5.0 - 6.0', '1.0 and up',
       '2.2 - 7.1.1', '5.0 - 7.1.1'], dtype=object)

Nothing out of the ordinary!

<h2>Finish</h2>

Finally, we save the dataframe into a new csv file

In [261]:
apps_df.to_csv("./data/googleplaystore_clean.csv",index=False)